In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

In [74]:
data = pd.read_csv('202310-divvy-tripdata.csv')
data = data[pd.to_numeric(data['start_station_id'], errors='coerce').notnull()]
data = data[pd.to_numeric(data['end_station_id'], errors='coerce').notnull()]
data = data.dropna(subset=['start_station_id', 'end_station_id'])

In [99]:

start = pd.DataFrame(data['start_station_id'].value_counts())
end = pd.DataFrame(data['end_station_id'].value_counts())
start = start.rename(columns={'count': 'startcount'})
start.head()

,startcount
start_station_id,
13022,2230
13300,1883
13216,1310
15544,1284
13008,1258


In [100]:
end = end.rename(columns={'count': 'endcount'})
end.head()

,endcount
end_station_id,
13022,2546
13300,1703
13008,1403
13216,1305
13042,1289


In [102]:
stations = pd.concat([start['startcount'], end['endcount']], axis=1)
stations['total'] = stations['startcount'] + stations['endcount']
stations.head(10)
#stations['total'] = stations['start_station_id'] + stations['end_station_id']
#stations.head(10)

,startcount,endcount,total
13022,2230.0,2546.0,4776.0
13300,1883.0,1703.0,3586.0
13216,1310.0,1305.0,2615.0
15544,1284.0,1114.0,2398.0
13008,1258.0,1403.0,2661.0
13042,1150.0,1289.0,2439.0
13217,1092.0,1067.0,2159.0
13011,1004.0,943.0,1947.0
638,988.0,1047.0,2035.0
13061,965.0,964.0,1929.0


In [103]:
stats = stations.sort_values('total', ascending=False).head(10)
stats = list(stats.index)
print(stats)

['13022', '13300', '13008', '13216', '13042', '15544', '13217', '638', '13011', '13061']


In [104]:
# take out just these top stations from the data
divd = data[(data['start_station_id'].isin(stats)) | (data['end_station_id'].isin(stats))]
divd.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
603,A1B0885683D4B4CC,classic_bike,2023-10-13 18:31:25,2023-10-13 18:54:00,Streeter Dr & Grand Ave,13022,Larrabee St & Webster Ave,13193,41.892278,-87.612043,41.921822,-87.644140,member
608,8BECFAB67F1FA81D,classic_bike,2023-10-22 17:44:50,2023-10-22 18:09:05,Streeter Dr & Grand Ave,13022,Larrabee St & Webster Ave,13193,41.892278,-87.612043,41.921822,-87.644140,casual
1042,BE2395919F04F1E0,electric_bike,2023-10-28 15:56:13,2023-10-28 16:15:34,Streeter Dr & Grand Ave,13022,Aberdeen St & Randolph St,18062,41.892319,-87.612234,41.884114,-87.654264,member
1198,8527F766D559425F,electric_bike,2023-10-28 15:55:45,2023-10-28 16:15:36,Streeter Dr & Grand Ave,13022,Aberdeen St & Randolph St,18062,41.892313,-87.612226,41.884114,-87.654264,member
1200,A4A3170B63E200BB,electric_bike,2023-10-23 18:26:25,2023-10-23 18:41:07,Streeter Dr & Grand Ave,13022,Larrabee St & Webster Ave,13193,41.892294,-87.612193,41.921822,-87.644140,member


In [105]:
print(divd.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')


In [106]:
# don't need ride_id, rideable_type, start_lat, start_lng, end_lat, end_lng, or member_casual
divd = divd[divd['started_at':'end_station_id']]
divd.head()

TypeError: cannot do slice indexing on Index with these indexers [started_at] of type str